In [2]:
import pandas as pd
import numpy as np

In [3]:
draft_path = 'resources/nfl_draft.csv'
draft_df = pd.read_csv(draft_path, encoding ='utf8')
draft_df

,Player_Id,Year,Rnd,Pick,Tm,Player,Pos,Position Standard,First4AV,Age,...,Rush_Yds,Rush_TDs,Rec,Rec_Yds,Rec_Tds,Tkl,Def_Int,Sk,College/Univ,Unnamed: 32
0,WinsJa00,2015,1.0,1,TAM,Jameis Winston,QB,QB,13,21.0,...,311.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,Florida St.,NaN
1,MariMa01,2015,1.0,2,TEN,Marcus Mariota,QB,QB,9,21.0,...,516.0,4.0,1.0,41.0,1.0,NaN,NaN,NaN,Oregon,NaN
2,FowlDa00,2015,1.0,3,JAX,Dante Fowler,OLB,LB,0,21.0,...,NaN,NaN,NaN,NaN,NaN,16.0,NaN,2.5,Florida,NaN
3,CoopAm00,2015,1.0,4,OAK,Amari Cooper,WR,WR,9,21.0,...,-3.0,0.0,134.0,1970.0,9.0,NaN,NaN,NaN,Alabama,NaN
4,ScheBr00,2015,1.0,5,WAS,Brandon Scherff,T,T,7,23.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Iowa,NaN
5,WillLe02,2015,1.0,6,NYJ,Leonard Williams,DE,DE,9,21.0,...,NaN,NaN,NaN,NaN,NaN,56.0,NaN,9,USC,NaN
6,WhitKe00,2015,1.0,7,CHI,Kevin White,WR,WR,0,23.0,...,9.0,0.0,19.0,187.0,0.0,NaN,NaN,NaN,West Virginia,NaN
7,BeasVi00,2015,1.0,8,ATL,Vic Beasley,OLB,LB,7,23.0,...,NaN,NaN,NaN,NaN,NaN,44.0,1.0,13.5,Clemson,NaN
8,FlowEr00,2015,1.0,9,NYG,Ereck Flowers,T,T,9,21.0,...,NaN,NaN,1.0,-11.0,0.0,NaN,NaN,NaN,Miami (FL),NaN
9,GurlTo01,2015,1.0,10,STL,Todd Gurley,RB,RB,8,21.0,...,1697.0,14.0,47.0,390.0,0.0,NaN,NaN,NaN,Georgia,NaN


In [4]:
draft_df_qb = draft_df.loc[draft_df['Pos']=='QB']
draft_df_qb = pd.DataFrame(draft_df_qb)
draft_df_qb.set_index('Player', inplace = True)
draft_df_qb= draft_df_qb[['Year','Rnd','Pick','Player_Id','Pos']]
draft_df_qb = draft_df_qb.loc[draft_df_qb['Year']>=2004]
draft_df_qb

,Year,Rnd,Pick,Player_Id,Pos
Player,,,,,
Jameis Winston,2015,1.0,1,WinsJa00,QB
Marcus Mariota,2015,1.0,2,MariMa01,QB
Garrett Grayson,2015,3.0,75,GrayGa00,QB
Sean Mannion,2015,3.0,89,MannSe00,QB
Bryce Petty,2015,4.0,103,PettBr01,QB
Brett Hundley,2015,5.0,147,HundBr00,QB
Trevor Siemian,2015,7.0,250,SiemTr00,QB
Blake Bortles,2014,1.0,3,BortBl00,QB
Johnny Manziel,2014,1.0,22,ManzJo00,QB


In [5]:
stats_path = 'resources/QBStats_all.csv'
stats_df = pd.read_csv(stats_path, encoding ='utf8')
stats_df.head()

,qb,att,cmp,yds,ypa,td,int,lg,sack,loss,rate,game_points,home_away,year
0,Boomer EsiasonB. Esiason,38,25,237.0,6.2,0,0,20,2.0,11.0,82.9,13,away,1996
1,Jim HarbaughJ. Harbaugh,25,16,196.0,7.8,2,1,35t,0.0,0.0,98.1,20,home,1996
2,Paul JustinP. Justin,8,5,53.0,6.6,0,0,30,1.0,11.0,81.8,20,home,1996
3,Jeff GeorgeJ. George,35,16,215.0,6.1,0,0,55,7.0,53.0,65.8,6,away,1996
4,Kerry CollinsK. Collins,31,17,198.0,6.4,2,0,30,4.0,12.0,95.9,29,home,1996


In [6]:
qb_stats_df = stats_df
qb_stats_df
names = qb_stats_df['qb'].tolist()
qb_list = []
for name in names:
    split_name = name.split(".")
    drop_second = split_name[0]
    clean_name = drop_second[:-1]
    qb_list.append(clean_name)
stats_df['Player'] = qb_list
stats_df.head()

,qb,att,cmp,yds,ypa,td,int,lg,sack,loss,rate,game_points,home_away,year,Player
0,Boomer EsiasonB. Esiason,38,25,237.0,6.2,0,0,20,2.0,11.0,82.9,13,away,1996,Boomer Esiason
1,Jim HarbaughJ. Harbaugh,25,16,196.0,7.8,2,1,35t,0.0,0.0,98.1,20,home,1996,Jim Harbaugh
2,Paul JustinP. Justin,8,5,53.0,6.6,0,0,30,1.0,11.0,81.8,20,home,1996,Paul Justin
3,Jeff GeorgeJ. George,35,16,215.0,6.1,0,0,55,7.0,53.0,65.8,6,away,1996,Jeff George
4,Kerry CollinsK. Collins,31,17,198.0,6.4,2,0,30,4.0,12.0,95.9,29,home,1996,Kerry Collins


In [7]:
qb_stats_df = stats_df.drop('qb',axis=1)
qb_stats_df = qb_stats_df.set_index('Player')

In [8]:
qb_stats_df.sort_values(by="year", ascending=False)

,att,cmp,yds,ypa,td,int,lg,sack,loss,rate,game_points,home_away,year
Player,,,,,,,,,,,,,
Colin Kaepernick,22,17,215.0,9.8,1,0,29,5.0,24.0,122.3,23,home,2016
Brock Osweiler,39,25,269.0,6.9,2,1,36,1.0,13.0,90.7,26,home,2016
Aaron Rodgers,56,39,326.0,5.8,3,0,25,2.0,23.0,102.2,26,home,2016
Brian Hoyer,11,4,49.0,4.5,0,0,25,0.0,0.0,50.9,10,away,2016
Matt Barkley,15,6,81.0,5.4,0,2,26,1.0,10.0,18.3,10,away,2016
Drew Stanton,1,1,12.0,12.0,0,0,12,0.0,0.0,116.7,28,home,2016
Carson Palmer,34,23,213.0,6.3,1,0,14,0.0,0.0,94.4,28,home,2016
Geno Smith,6,4,31.0,5.2,0,1,17,1.0,8.0,39.6,3,away,2016
Ryan Fitzpatrick,31,16,174.0,5.6,0,1,36,0.0,0.0,55.0,3,away,2016


In [10]:
qb_grpd = qb_stats_df.groupby(['Player'])